In [1]:
# coding=utf-8

import sys
import json
import base64
import time

IS_PY3 = sys.version_info.major == 3

if IS_PY3:
    from urllib.request import urlopen
    from urllib.request import Request
    from urllib.error import URLError
    from urllib.parse import urlencode
    timer = time.perf_counter
else:
    from urllib2 import urlopen
    from urllib2 import Request
    from urllib2 import URLError
    from urllib import urlencode
    if sys.platform == "win32":
        timer = time.clock
    else:
        # On most other platforms the best timer is time.time()
        timer = time.time

API_KEY = '8ODiGCupWIaOpRquTpa3i9UA'
SECRET_KEY = 'XoEsmYsn4vVjcnOSh70eaR8CreeTOdCI'

CUID = '123456PYTHON'
# 采样率
RATE = 16000  # 固定值

# 普通版

DEV_PID = 1537  # 1537 表示识别普通话，使用输入法模型。根据文档填写PID，选择语言及识别模型
ASR_URL = 'http://vop.baidu.com/server_api'
SCOPE = 'audio_voice_assistant_get'  # 有此scope表示有asr能力，没有请在网页里勾选，非常旧的应用可能没有

class DemoError(Exception):
    pass


"""  TOKEN start """

TOKEN_URL = 'http://aip.baidubce.com/oauth/2.0/token'


def fetch_token():
    params = {'grant_type': 'client_credentials',
              'client_id': API_KEY,
              'client_secret': SECRET_KEY}
    post_data = urlencode(params)
    if (IS_PY3):
        post_data = post_data.encode( 'utf-8')
    req = Request(TOKEN_URL, post_data)
    try:
        f = urlopen(req)
        result_str = f.read()
    except URLError as err:
        print('token http response http code : ' + str(err.code))
        result_str = err.read()
    if (IS_PY3):
        result_str =  result_str.decode()

    print(result_str)
    result = json.loads(result_str)
    print(result)
    if ('access_token' in result.keys() and 'scope' in result.keys()):
        print(SCOPE)
        if SCOPE and (not SCOPE in result['scope'].split(' ')):  # SCOPE = False 忽略检查
            raise DemoError('scope is not correct')
        print('SUCCESS WITH TOKEN: %s  EXPIRES IN SECONDS: %s' % (result['access_token'], result['expires_in']))
        return result['access_token']
    else:
        raise DemoError('MAYBE API_KEY or SECRET_KEY not correct: access_token or scope not found in token response')

"""  TOKEN end """

if __name__ == '__main__':
    
    import os
    work_dir=r'D:\Graduation_project\audio\Release_Roadshow'
    audio_dirs=os.listdir(work_dir)
    for audio_dir in audio_dirs:
        os.makedirs(os.path.join(work_dir,audio_dir,'text'))
        section_dir=os.path.join(work_dir,audio_dir,'section')
        file_dirs=os.listdir(section_dir)
        for file_dir in file_dirs:
            input_file_dir=os.path.join(work_dir,audio_dir,'section',file_dir)
            # 需要识别的文件
            AUDIO_FILE = input_file_dir  # 只支持 pcm/wav/amr 格式，极速版额外支持m4a 格式
            # 文件格式
            FORMAT = AUDIO_FILE[-3:]  # 文件后缀只支持 pcm/wav/amr 格式，极速版额外支持m4a 格式
    
            token = fetch_token()

            speech_data = []
            with open(AUDIO_FILE, 'rb') as speech_file:
                speech_data = speech_file.read()

            length = len(speech_data)
            if length == 0:
                raise DemoError('file %s length read 0 bytes' % AUDIO_FILE)
            speech = base64.b64encode(speech_data)
            if (IS_PY3):
                speech = str(speech, 'utf-8')
            params = {'dev_pid': DEV_PID,
                 #"lm_id" : LM_ID,    #测试自训练平台开启此项
                  'format': FORMAT,
                  'rate': RATE,
                  'token': token,
                  'cuid': CUID,
                  'channel': 1,
                  'speech': speech,
                  'len': length
                  }
            post_data = json.dumps(params, sort_keys=False)
            # print post_data
            req = Request(ASR_URL, post_data.encode('utf-8'))
            req.add_header('Content-Type', 'application/json')
            try:
                begin = timer()
                f = urlopen(req)
                result_str = f.read()
                print ("Request time cost %f" % (timer() - begin))
            except URLError as err:
                print('asr http response http code : ' + str(err.code))
                result_str = err.read()

            if (IS_PY3):
                result_str = str(result_str, 'utf-8')
            print(result_str)
            output_file_dir=os.path.join(work_dir,audio_dir,'text',file_dir)
            with open(output_file_dir[:-3]+'txt',"w") as of:
                of.write(result_str)

{"refresh_token":"25.dbcf09356befa7096cb119a06dfbaec5.315360000.1964400018.282335-25879090","expires_in":2592000,"session_key":"9mzdWWuMOzP9Co+3WIUw4RqaqQaAgXllsNAV41kay+J6i3\/6yXHNpbJrVLztT4CqDZ9AE3J\/moB44L\/4ayDDgx83X14uNA==","access_token":"24.b1b1107629f74dce6c38287c1748906d.2592000.1651632018.282335-25879090","scope":"audio_voice_assistant_get brain_enhanced_asr audio_tts_post brain_speech_realtime public brain_all_scope brain_asr_async wise_adapt lebo_resource_base lightservice_public hetu_basic lightcms_map_poi kaidian_kaidian ApsMisTest_Test\u6743\u9650 vis-classify_flower lpq_\u5f00\u653e cop_helloScope ApsMis_fangdi_permission smartapp_snsapi_base smartapp_mapp_dev_manage iop_autocar oauth_tp_app smartapp_smart_game_openapi oauth_sessionkey smartapp_swanid_verify smartapp_opensource_openapi smartapp_opensource_recapi fake_face_detect_\u5f00\u653eScope vis-ocr_\u865a\u62df\u4eba\u7269\u52a9\u7406 idl-video_\u865a\u62df\u4eba\u7269\u52a9\u7406 smartapp_component smartapp_searc

Request time cost 2.390624
{"corpus_no":"7082572974259058819","err_msg":"success.","err_no":0,"result":["我们非常高兴参加今天的网上交流活动，首先，我谨代表华泰联合证券对所有参与华南疫苗网上路演的嘉宾和投资者表示热烈的欢迎和衷心的感谢，华兰疫苗作为一家集疫苗研发生产为一体的创新型生物制药企业，在流感疫苗的研发，生产和销售领域占据领先地位，2009年，公司研制出全球首批甲型h1n1流感病毒裂解疫苗，该产品获得国家科学技术部，国家质量监督检验检验检疫总局等机构联合颁发的国家重点新产品证书，2018年华了疫苗。"],"sn":"436204198561649040024"}

{"refresh_token":"25.92ab9ca6d3ac575914350bd77d31c02c.315360000.1964400026.282335-25879090","expires_in":2592000,"session_key":"9mzdCy1AlEFPmgy6s513V4Bupuf+XPgW9D4AHZv\/Vv03bDzfqPs1iLzBy\/s+5g7JbKgW9\/kRYWbHhswnkU0tgIsSnVuAhA==","access_token":"24.05d2e327c1f63a35b95551e34bd94d13.2592000.1651632026.282335-25879090","scope":"audio_voice_assistant_get brain_enhanced_asr audio_tts_post brain_speech_realtime public brain_all_scope brain_asr_async wise_adapt lebo_resource_base lightservice_public hetu_basic lightcms_map_poi kaidian_kaidian ApsMisTest_Test\u6743\u9650 vis-classify_flower lpq_\u5f00\u653e cop_helloScope ApsMis_fangdi_permission smartapp_snsa

Request time cost 0.919211
{"corpus_no":"7082573008357665567","err_msg":"success.","err_no":0,"result":["更多情况，健康中国预防当先，随着健康中国的全面推进，预防为主的公共卫生体制日益健全，华兰生物疫苗股份有限公司依托母公司30年的生物医药行业经验，凭借自身不断创新和发展取得了国内外多个行业第一，承担了国家863计划项目，国家重点产业化项目等多项国家，省部级科技攻关项目已建成的人。"],"sn":"86037548411649040032"}

{"refresh_token":"25.c39638a8bc3384a76d409d18c77f5e35.315360000.1964400033.282335-25879090","expires_in":2592000,"session_key":"9mzdCuCvcyItPokEzuWfBW0CzGDV9VrlWLYGKqSKBE4pkZHKc1Y7NgTw73Ue917TxJEvJ8sbgIQ9kIC206p9XEcgk+Jxgw==","access_token":"24.1dcb95f1b363dd5fe3f83d95eeba27cf.2592000.1651632033.282335-25879090","scope":"audio_voice_assistant_get brain_enhanced_asr audio_tts_post brain_speech_realtime public brain_all_scope brain_asr_async wise_adapt lebo_resource_base lightservice_public hetu_basic lightcms_map_poi kaidian_kaidian ApsMisTest_Test\u6743\u9650 vis-classify_flower lpq_\u5f00\u653e cop_helloScope ApsMis_fangdi_permission smartapp_snsapi_base smartapp_mapp_dev_manage iop_autocar oauth_tp_app smartapp_

Request time cost 1.527428
{"corpus_no":"7082573034325748424","err_msg":"success.","err_no":0,"result":["动细胞及疫苗通用制备工艺技术平台及其机制疫苗研发生产平台腺病毒载体疫苗研发生产平台及生物安全三级实验室，高生物安全风险生产平台，为公司未来的快速发展奠定了基础，自公司成立以来，华南人秉承锐意拼搏，勇往直前的精神，勤耕不辍，砥砺前行，长风破浪会有时，直挂云帆济沧海。"],"sn":"996646968641649040038"}

{"refresh_token":"25.a88b5b4e3e4c12bf0ae361b3c4af4d9b.315360000.1964400039.282335-25879090","expires_in":2592000,"session_key":"9mzdDc7sIyT\/jyzmRVX+k4emduPOrmgdqxk3BiDjKgRen3SwJR223IshaJQqWDu\/0Ykaw+kuRSY0QTDQI+GgAAr4jaK+rA==","access_token":"24.6891141f652ff43a163c98bc7329ef54.2592000.1651632039.282335-25879090","scope":"audio_voice_assistant_get brain_enhanced_asr audio_tts_post brain_speech_realtime public brain_all_scope brain_asr_async wise_adapt lebo_resource_base lightservice_public hetu_basic lightcms_map_poi kaidian_kaidian ApsMisTest_Test\u6743\u9650 vis-classify_flower lpq_\u5f00\u653e cop_helloScope ApsMis_fangdi_permission smartapp_snsapi_base smartapp_mapp_dev_manage iop_autocar oauth_tp_app smartapp_smar

Request time cost 1.980750
{"corpus_no":"7082573064232097712","err_msg":"success.","err_no":0,"result":["能力，人才优势等等等等，接下来我们就来了解一下华兰生物疫苗股份有限公司有什么样的投资亮点嗯嗯嗯嗯。"],"sn":"6955939911649040045"}

{"refresh_token":"25.5c362242c36594a7892ddb90c938187a.315360000.1964400047.282335-25879090","expires_in":2592000,"session_key":"9mzdDFP9FwUhvTUf8lY19EjvcNebCj7iuHbgiQpKhqMUoEkf6p2OipUV4rb2XwJieOBoJ6lhpW858VDCz8rOy7c\/1s0LRQ==","access_token":"24.7fa57efad2465504b92f88d0a502dc29.2592000.1651632047.282335-25879090","scope":"audio_voice_assistant_get brain_enhanced_asr audio_tts_post brain_speech_realtime public brain_all_scope brain_asr_async wise_adapt lebo_resource_base lightservice_public hetu_basic lightcms_map_poi kaidian_kaidian ApsMisTest_Test\u6743\u9650 vis-classify_flower lpq_\u5f00\u653e cop_helloScope ApsMis_fangdi_permission smartapp_snsapi_base smartapp_mapp_dev_manage iop_autocar oauth_tp_app smartapp_smart_game_openapi oauth_sessionkey smartapp_swanid_verify smartapp_opensource_openapi sma

...

Request time cost 0.881883
{"corpus_no":"7082573639915225481","err_msg":"success.","err_no":0,"result":["嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯。"],"sn":"261765165481649040179"}

{"refresh_token":"25.e44b782716cc72ed7ffb6b12919af81d.315360000.1964400180.282335-25879090","expires_in":2592000,"session_key":"9mzdX7nm+kp5mOyrlFrJvdqqFZua0cutQriteDeEx4\/Sk0hi43ljnJUfxcIYm+kn8GwpJYFwbqkT6yxwtgUp8OccVxZf7g==","access_token":"24.dc50b0a3c8fc07416f04d63d29413b8b.2592000.1651632180.282335-25879090","scope":"audio_voice_assistant_get brain_enhanced_asr audio_tts_post brain_speech_realtime public brain_all_scope brain_asr_async wise_adapt lebo_resource_base lightservice_public hetu_basic lightcms_map_poi kaidian_kaidian ApsMisTest_Test\u6743\u9650 vis-classify_flower lpq_\u5f00\u653e cop_helloScope ApsMis_fangdi_permission smartapp_snsapi_base smartapp_mapp_dev_manage iop_autocar oauth_tp_app smartapp_smart_game_openapi oauth_sessionkey smartapp_swanid_verify smartapp_opensource_openapi smartapp_opensource_recapi fake_fac

Request time cost 1.009602
{"corpus_no":"7082573674272846312","err_msg":"success.","err_no":0,"result":["嗯啊嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯。"],"sn":"927294743421649040187"}

{"refresh_token":"25.0fd93b4e8ba2ca41f2a556fc7486e2bd.315360000.1964400188.282335-25879090","expires_in":2592000,"session_key":"9mzdDorDMZcWuGLTtbGDy\/O+Ya\/eTAuDYQk66lyte35uzBpQ149rExeC6CEe2qhB2X6biooVet297qeZSot\/fDMsOY3Z4A==","access_token":"24.58574c8020453849ab6eca3f46ebb58d.2592000.1651632188.282335-25879090","scope":"audio_voice_assistant_get brain_enhanced_asr audio_tts_post brain_speech_realtime public brain_all_scope brain_asr_async wise_adapt lebo_resource_base lightservice_public hetu_basic lightcms_map_poi kaidian_kaidian ApsMisTest_Test\u6743\u9650 vis-classify_flower lpq_\u5f00\u653e cop_helloScope ApsMis_fangdi_permission smartapp_snsapi_base smartapp_mapp_dev_manage iop_autocar oauth_tp_app smartapp_smart_game_openapi oauth_sessionkey smartapp_swanid_verify smartapp_opensource_openapi smartapp_opensource_recapi fa

Request time cost 0.837585
{"corpus_no":"7082573695757086175","err_msg":"success.","err_no":0,"result":["嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯嗯。"],"sn":"430083686401649040192"}

{"refresh_token":"25.50c22990a1eb81ff2272c03e7b73cd8a.315360000.1964400192.282335-25879090","expires_in":2592000,"session_key":"9mzdAqBLAIG5kYi\/8+VBNIWTJFd7Z7afPJkXK7FSLAHDrvYq15ube5n+oEUD9uxlJeXQ3jJPwQ5CWiWsfHvhEJTSp75+Qw==","access_token":"24.9ba951e0dd2dd6c05a7766a6fbfee0cd.2592000.1651632192.282335-25879090","scope":"audio_voice_assistant_get brain_enhanced_asr audio_tts_post brain_speech_realtime public brain_all_scope brain_asr_async wise_adapt lebo_resource_base lightservice_public hetu_basic lightcms_map_poi kaidian_kaidian ApsMisTest_Test\u6743\u9650 vis-classify_flower lpq_\u5f00\u653e cop_helloScope ApsMis_fangdi_permission smartapp_snsapi_base smartapp_mapp_dev_manage iop_autocar oauth_tp_app smartapp_smart_game_openapi oauth_sessionkey smartapp_swanid_verify smartapp_opensource_openapi smartapp_opensource_recapi fake_face

Request time cost 1.743088
{"corpus_no":"7082573717184194088","err_msg":"success.","err_no":0,"result":["高兴今天能够通过网络与各位投资者朋友就15检测，首次公开发行股票，现在创业板上市进行实时在线交流，在此，我谨代表公司向参加今天交流活动的各位朋友表示最热烈的欢迎，对长期以来关心支持师傅检测的投资者和各界朋友表示最诚挚的谢意，食品检测深根土壤和地下水检测领域多年检测范围包括土壤，水质，气体，固体废物和农业等，涵盖从农田到。"],"sn":"599887929551649040197"}

{"refresh_token":"25.5dfc65889725c4114c767d0b04027e4b.315360000.1964400199.282335-25879090","expires_in":2592000,"session_key":"9mzdWuUUck2C+qtWL8xfT+EzzcZIE3m9GCCNCMwkBw\/AZLbKeSAgRlLgMYEZ0TuX\/bUcoicpvxkQra6n33FhqS2CjRxJJg==","access_token":"24.a99aa195cbbdd61424b6769117012eb2.2592000.1651632199.282335-25879090","scope":"audio_voice_assistant_get brain_enhanced_asr audio_tts_post brain_speech_realtime public brain_all_scope brain_asr_async wise_adapt lebo_resource_base lightservice_public hetu_basic lightcms_map_poi kaidian_kaidian ApsMisTest_Test\u6743\u9650 vis-classify_flower lpq_\u5f00\u653e cop_helloScope ApsMis_fangdi_permission smartapp_snsapi_base smartapp_mapp_dev_manage iop_autoc

Request time cost 1.661247
{"corpus_no":"7082573755784675139","err_msg":"success.","err_no":0,"result":["餐桌等于环境及食品安全相关的检测业务，公司于2013年建立第一家土壤和地下水检测实验室，自成立以来，公司积极把握政策机遇，快速扩张，在长三角，京津冀，珠三角等重点地区相继建立多家实验室及联络处，业务规模快速增长，十部检测的创始管理团队是中国最早从事土壤和地下水污染防治相关产业的专家，也是国内最早从事土壤和地下水检测的第三方检测机构之一，公司积极不。"],"sn":"706139267681649040205"}

{"refresh_token":"25.2ecdcde065d2eae1e9b6536edb2dac94.315360000.1964400207.282335-25879090","expires_in":2592000,"session_key":"9mzdWEV9TLvVGOnviGmuXjp9WD9YFpIIfl\/iQeFjdzkp+7XSAEBOeXtWIU5vr6HJutRi8A4ZVHARBvnugMIMOmowS6xC4w==","access_token":"24.c5bc898810ff7d3c210f5f70da68213c.2592000.1651632207.282335-25879090","scope":"audio_voice_assistant_get brain_enhanced_asr audio_tts_post brain_speech_realtime public brain_all_scope brain_asr_async wise_adapt lebo_resource_base lightservice_public hetu_basic lightcms_map_poi kaidian_kaidian ApsMisTest_Test\u6743\u9650 vis-classify_flower lpq_\u5f00\u653e cop_helloScope ApsMis_fangdi_permission smartapp_snsapi_base smartapp_mapp_dev_man